___

<a href='http://www.pieriandata.com'> <img src='../Pierian_Data_Logo.png' /></a>
___

# Latent Dirichlet Allocation

## Data

We will be using articles from NPR (National Public Radio), obtained from their website [www.npr.org](http://www.npr.org)

In [0]:
import pandas as pd

In [0]:
npr = pd.read_csv('/dbfs/FileStore/ds_data_files/npr.csv')

In [0]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [0]:
print(npr['Article'][100])

There are more than 80, 000 educational apps in Apple’s app store. It seems like a great way to encourage brain development and make your little one the smartest baby genius. But just sticking a tablet in your kid’s hands might not be as helpful. Sure, use the app. But it’s not a babysitter  —   you’ve got to help them use it, too. Several recent studies have looked at how young children learn from touchscreens. One study, published in Child Development, compared how   and    learned to build a   puzzle. Some children learned how to assemble the puzzle from a ”ghost demonstration”  —   meaning that, initially, the pieces moved by themselves on the tablet to show how it works. A lot of apps that are intended for young children often have some element of this ghost demonstration: Pieces move on their own or objects will move them. Other children had a person sitting next to them to move the puzzle pieces on the tablet. After they watched the demonstration, both groups of children were as

In [0]:
len(npr)

Out[8]: 11992

Notice how we don't have the topic of the articles! Let's use LDA to attempt to figure out clusters of the articles.

## Preprocessing

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

**`max_df`**` : float in range [0.0, 1.0] or int, default=1.0`<br>
When building the vocabulary ignore terms that have a document frequency strictly higher than the given threshold (corpus-specific stop words). If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

**`min_df`**` : float in range [0.0, 1.0] or int, default=1`<br>
When building the vocabulary ignore terms that have a document frequency strictly lower than the given threshold. This value is also called cut-off in the literature. If float, the parameter represents a proportion of documents, integer absolute counts. This parameter is ignored if vocabulary is not None.

In [0]:
cv = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')

In [0]:
dtm = cv.fit_transform(npr['Article'])

In [0]:
dtm

Out[13]: <11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

## LDA

In [0]:
from sklearn.decomposition import LatentDirichletAllocation

In [0]:
LDA = LatentDirichletAllocation(n_components=7,random_state=42)

In [0]:
# This can take awhile, we're dealing with a large amount of documents!
LDA.fit(dtm)

Out[16]: LatentDirichletAllocation(n_components=7, random_state=42)

## Showing Stored Words
Get vocabulary of words

In [0]:
len(cv.get_feature_names()) # the DTM has 54777 columns

Out[17]: 54777

In [0]:
type(cv.get_feature_names())  # this is just a list of vocab words in documents

Out[19]: list

In [0]:
import random

for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names()[random_word_id])

offenses
welsh
battled
saran
rebutting
sans
bureaus
derailed
converting
bail


In [0]:
for i in range(10):
    random_word_id = random.randint(0,54776)
    print(cv.get_feature_names()[random_word_id])

foremothers
mocoa
ellroy
liron
ally
discouraged
utterance
provo
videgaray
archivist


### Showing Top Words Per Topic
Get the topics

In [0]:
len(LDA.components_)

Out[22]: 7

In [0]:
LDA.components_  # probabilities for each word

Out[23]: array([[8.64332806e+00, 2.38014333e+03, 1.42900522e-01, ...,
        1.43006821e-01, 1.42902042e-01, 1.42861626e-01],
       [2.76191749e+01, 5.36394437e+02, 1.42857148e-01, ...,
        1.42861973e-01, 1.42857147e-01, 1.42906875e-01],
       [7.22783888e+00, 8.24033986e+02, 1.42857148e-01, ...,
        6.14236247e+00, 2.14061364e+00, 1.42923753e-01],
       ...,
       [3.11488651e+00, 3.50409655e+02, 1.42857147e-01, ...,
        1.42859912e-01, 1.42857146e-01, 1.42866614e-01],
       [4.61486388e+01, 5.14408600e+01, 3.14281373e+00, ...,
        1.43107628e-01, 1.43902481e-01, 2.14271779e+00],
       [4.93991422e-01, 4.18841042e+02, 1.42857151e-01, ...,
        1.42857146e-01, 1.43760101e-01, 1.42866201e-01]])

In [0]:
LDA.components_.shape  # 7 topics X 54777 vocabulary words

In [0]:
len(LDA.components_[0])

Out[25]: 54777

In [0]:
single_topic = LDA.components_[0]

In [0]:
# Returns the indices that would sort this array from lowest probability value to highest probability value
single_topic.argsort()

Out[27]: array([ 2475, 18302, 35285, ..., 22673, 42561, 42993])

In [0]:
# Word least representative of this topic
single_topic[18302]

0.14285714309286987

In [0]:
# Word most representative of this topic
single_topic[42993]

6247.245510521082

In [0]:
# Top 10 words for this topic:
single_topic.argsort()[-10:]

Out[28]: array([33390, 36310, 21228, 10425, 31464,  8149, 36283, 22673, 42561,
       42993])

In [0]:
top_word_indices = single_topic.argsort()[-10:]

In [0]:
for index in top_word_indices:
    print(cv.get_feature_names()[index])

new
percent
government
company
million
care
people
health
said
says


These look like business articles perhaps... Let's confirm by using .transform() on our vectorized articles to attach a label number. But first, let's view all the 10 topics found.

In [0]:
for index,topic in enumerate(LDA.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['companies', 'money', 'year', 'federal', '000', 'new', 'percent', 'government', 'company', 'million', 'care', 'people', 'health', 'said', 'says']


THE TOP 15 WORDS FOR TOPIC #1
['military', 'house', 'security', 'russia', 'government', 'npr', 'reports', 'says', 'news', 'people', 'told', 'police', 'president', 'trump', 'said']


THE TOP 15 WORDS FOR TOPIC #2
['way', 'world', 'family', 'home', 'day', 'time', 'water', 'city', 'new', 'years', 'food', 'just', 'people', 'like', 'says']


THE TOP 15 WORDS FOR TOPIC #3
['time', 'new', 'don', 'years', 'medical', 'disease', 'patients', 'just', 'children', 'study', 'like', 'women', 'health', 'people', 'says']


THE TOP 15 WORDS FOR TOPIC #4
['voters', 'vote', 'election', 'party', 'new', 'obama', 'court', 'republican', 'campaign', 'people', 'state', 'president', 'clinton', 'said', 'trump']


THE TOP 15 WORDS FOR TOPIC #5
['years', 'going', 've', 'life', 'don', 'new', 'way', 'music', 'really', 'time', 'know', 'think',

### Attaching Discovered Topic Labels to Original Articles
Probability of document belonging to each topic

In [0]:
dtm

Out[34]: <11992x54777 sparse matrix of type '<class 'numpy.int64'>'
	with 3033388 stored elements in Compressed Sparse Row format>

In [0]:
dtm.shape

Out[35]: (11992, 54777)

In [0]:
len(npr)

Out[36]: 11992

In [0]:
topic_results = LDA.transform(dtm)

In [0]:
topic_results.shape  # distribution of topic probabilities for each document

Out[38]: (11992, 7)

In [0]:
topic_results[0]

Out[39]: array([1.61040465e-02, 6.83341493e-01, 2.25376318e-04, 2.25369288e-04,
       2.99652737e-01, 2.25479379e-04, 2.25497980e-04])

In [0]:
topic_results[0].round(2)

Out[40]: array([0.02, 0.68, 0.  , 0.  , 0.3 , 0.  , 0.  ])

In [0]:
topic_results[0].argmax()

Out[41]: 1

This means that our model thinks that the first article belongs to topic #1.

### Combining with Original Data

In [0]:
npr.head()

,Article
0,"In the Washington of 2016, even when the polic..."
1,Donald Trump has used Twitter — his prefe...
2,Donald Trump is unabashedly praising Russian...
3,"Updated at 2:50 p. m. ET, Russian President Vl..."
4,"From photography, illustration and video, to d..."


In [0]:
topic_results.argmax(axis=1)

Out[43]: array([1, 1, 1, ..., 3, 4, 0])

In [0]:
npr['Topic'] = topic_results.argmax(axis=1)

In [0]:
npr.head(10)

,Article,Topic
0,"In the Washington of 2016, even when the polic...",1
1,Donald Trump has used Twitter — his prefe...,1
2,Donald Trump is unabashedly praising Russian...,1
3,"Updated at 2:50 p. m. ET, Russian President Vl...",1
4,"From photography, illustration and video, to d...",2
5,I did not want to join yoga class. I hated tho...,3
6,With a who has publicly supported the debunk...,3
7,"I was standing by the airport exit, debating w...",2
8,"If movies were trying to be more realistic, pe...",3
9,"Eighteen years ago, on New Year’s Eve, David F...",2


In [0]:
npr['Topic'].value_counts()*100.0/len(npr)

Out[47]: 5    20.496998
1    16.711141
4    16.202468
2    15.326885
3    12.383256
0    10.948966
6     7.930287
Name: Topic, dtype: float64

## Great work!